<a href="REF: https://github.com/GoogleCloudPlatform/df-ml-anomaly-detection/blob/master/README.md">Reference1<a/>  <br>
<a href="https://faker.readthedocs.io/en/master/">Reference2<a/> 

In [ ]:
with beam.Pipeline() as pipeline:
    total_unique_elements = (
        pipeline
        | 'Create produce' >> beam.Create(
            [{"name":"idris","salary":500,"test":5}, {"name":"idris","salary":500,"test":5}, {"name":"idris","salary":500,"test":6}, {"name":"ade","salary":900,"test":5}])
        | 'row' >> beam.Map(lambda row: beam.Row(name=row['name'],salary=row["salary"],test=row["test"]))
        | 'Pair with 1' >> beam.GroupBy("name")\
        .aggregate_field(
            "name", count_distint, "count"
        ).aggregate_field(
            "test", beam.Distinct(), "test"
        )
        #     "name", beam.combiners.Iterable(), "count").aggregate_field(
        #       "salary", beam.combiners.MeanCombineFn(), "avg_salary").aggregate_field(
        #         "salary", sum, "total_salary")
        | beam.Map(print)
        )

In [ ]:
pipeline = beam.Pipeline()
raw_events = (
    pipeline
    | 'Read from PubSub' >> beam.Create(['{"foo": "bar"}' for _ in range(10)])
    | 'Parse events' >> beam.ParDo(EventParser()).with_outputs('valid', 'invalid')
)

(raw_events.invalid 
 |'Log invalid events' >> beam.io.fileio.WriteToFiles("data/invalid_events",shards=2,max_writers_per_bundle=2))

(raw_events.valid 
 | "time" >> beam.Map(lambda x: beam.window.TimestampedValue(x, time.time()))
 | 'Add timestamps' >> beam.ParDo(AddTimestamp())
 | "add key" >> beam.Map(print))

pipeline.run()

In [129]:

import time
import json
import logging
import random
import apache_beam as beam
from utils.custom import NetworkPool, UserObject, JsonEvent
from apache_beam.transforms.periodicsequence import PeriodicImpulse
from apache_beam.options.pipeline_options import PipelineOptions
        
def to_json(event):
    return json.dumps(event).encode('utf-8')        

class AgggregateEvent(beam.DoFn):
    def process(self, element):
        
        network = self.network_pool.get_network()
        user = self.user_obj.get_user()
        num_events = random.randint(5, self.max_events_per_session)
        for _ in range(num_events):
            event = JsonEvent.generate(user, network)
            yield event
                        
def run(args, beam_args):
    options = PipelineOptions(beam_args, save_main_session=True, streaming=True)
    pipeline = beam.Pipeline(options=options)
    (
        pipeline
        | "Trigger" >> PeriodicImpulse(start_timestamp=time.time(), fire_interval=(60/args.qps))
        | "Generate Events" >> beam.ParDo(EventGenerator())
        | "JSONIFY" >> beam.Map(to_json)
        | "Write to PubSub" >> beam.io.WriteToPubSub(args.topic)
    )
    return pipeline.run()

In [ ]:
# raw data

{
  "subscriberId":(11111111111,99999999999)
  "srcIP": "ipv4"
  "dstIP": "subnet"
  "srcPort": (1000,5000)
  "dstPort": (1000,5000)
  "txBytes": (0,1000000000)
  "rxBytes": (0,1000000000)
  "tcpFlag": (0,65)
  "startTime": "timestamp"
  "endTime": "timestamp"
  "protocolName": "tcp|udp|http"
  "protocolNumber": (0,255)
  "geoCountry": "country"
  "geoCity": "city"
  "latitude": (0,90)
  "longitude": (0,180)
}

# transformed data
{
  "transaction_time": "2019-10-27 23:22:17.848000",
  "subscriber_id": "100",
  "dst_subnet": "12.0.1.2/22",
  "number_of_unique_ips": "1",
  "number_of_unique_ports": "1",
  "number_of_records": "2",
  "max_tx_bytes": "15",
  "min_tx_bytes": "10",
  "avg_tx_bytes": "12.5",
  "max_rx_bytes": "40",
  "min_rx_bytes": "40",
  "avg_rx_bytes": "40.0",
  "max_duration": "100",
  "min_duration": "9",
  "avg_duration": "54.5"
}

# model features
{  "number_of_records": "2",
  "max_tx_bytes": "15",
  "min_tx_bytes": "10",
  "avg_tx_bytes": "12.5",
  "max_rx_bytes": "40",
  "min_rx_bytes": "40",
  "avg_rx_bytes": "40.0",
  "max_duration": "100",
  "min_duration": "9",
  "avg_duration": "54.5"
  }

In [ ]:
# python3 pipeline.py --streaming --project "electric-armor-395015" --topic "projects/electric-armor-395015/topics/test" --qps 10 --runner "DataflowRunner" --region europe-west2 temp_location gs://my_terraform_state_bucket/temp staging_location gs://my_terraform_state_bucket/staging

In [ ]:
# gcloud pubsub subscriptions pull test-sub --auto-ack --limit 1